- https://github.com/qqwweee/keras-yolo3
    - `convert.py`

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Config" data-toc-modified-id="Config-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Config</a></span><ul class="toc-item"><li><span><a href="#Path" data-toc-modified-id="Path-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Path</a></span></li></ul></li><li><span><a href="#yolov3.weight" data-toc-modified-id="yolov3.weight-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>yolov3.weight</a></span></li></ul></div>

## Config

In [34]:
import os
import configparser
import pathlib
import collections
import io

import numpy as np

### Path

In [19]:
HOME_Path = pathlib.Path(os.getcwd())

config_Path = HOME_Path / "yolov3.cfg"
assert config_Path.exists
assert config_Path.name.endswith(".cfg"), "{} is not a .cfg file".format(config_Path)

weights_Path = HOME_Path / "yolov3.weights"
assert weights_Path.exists
assert weights_Path.name.endswith(".weights"), "{} is not a .weights file".format(weights_Path)

## yolov3.weight

In [27]:
weights_file = open(file=str(weights_Path), mode="rb")

In [35]:
major, minor, revision = np.ndarray(shape=(3, ), dtype='int32', buffer=weights_file.read(12))

if (major*10+minor)>=2 and major<1000 and minor<1000:
    seen = np.ndarray(shape=(1,), dtype='int64', buffer=weights_file.read(8))
else:
    seen = np.ndarray(shape=(1,), dtype='int32', buffer=weights_file.read(4))

print("Weights Header\n" +
      "major    : {}\n".format(major) +
      "minor    : {}\n".format(minor) +
      "revision : {}\n".format(revision) +
      "seen     : {}".format(seen))

Weights Header
major    : 0
minor    : 2
revision : 0
seen     : [32013312]


In [36]:
def unique_config_sections(config_file):
    """
    Convert all config sections to have unique names.
    Adds unique suffixes to config sections for compability with configparser.
    """
    # [defaultdict の 利用 - Python defaultdict の使い方 - Qiita](https://qiita.com/xza/items/72a1b07fcf64d1f4bdb7#defaultdict-%E3%81%AE-%E5%88%A9%E7%94%A8)
    section_counters = collections.defaultdict(int)  # defaultdictは関数でdictのvalueを初期化
    output_stream = io.StringIO()
    with open(config_file) as fin:
        for line in fin:
            if line.startswith('['):
                section = line.strip().strip('[]')
                _section = section + '_' + str(section_counters[section])
                section_counters[section] += 1
                line = line.replace(section, _section)
            output_stream.write(line)
    output_stream.seek(0)
    return output_stream

In [37]:
print('Parsing Darknet config.')
unique_config_file = unique_config_sections(config_file=str(config_Path))

cfg_parser = configparser.ConfigParser()
cfg_parser.read_file(f=unique_config_file)

Parsing Darknet config.


In [48]:
weight_decay = float(cfg_parser['net_0']['decay']) if 'net_0' in cfg_parser.sections() else 5e-4

In [49]:
count = 0
out_index = []
for section in cfg_parser.sections():
    print('Parsing section {}'.format(section))

Parsing section net_0
Parsing section convolutional_0
Parsing section convolutional_1
Parsing section convolutional_2
Parsing section convolutional_3
Parsing section shortcut_0
Parsing section convolutional_4
Parsing section convolutional_5
Parsing section convolutional_6
Parsing section shortcut_1
Parsing section convolutional_7
Parsing section convolutional_8
Parsing section shortcut_2
Parsing section convolutional_9
Parsing section convolutional_10
Parsing section convolutional_11
Parsing section shortcut_3
Parsing section convolutional_12
Parsing section convolutional_13
Parsing section shortcut_4
Parsing section convolutional_14
Parsing section convolutional_15
Parsing section shortcut_5
Parsing section convolutional_16
Parsing section convolutional_17
Parsing section shortcut_6
Parsing section convolutional_18
Parsing section convolutional_19
Parsing section shortcut_7
Parsing section convolutional_20
Parsing section convolutional_21
Parsing section shortcut_8
Parsing section con

In [50]:
for section in cfg_parser.sections()[:5]:
    print("=== Parsing section : {} ===".format(section))
    #===============
    if section.startswith('convolutional'):
        filters = int(cfg_parser[section]['filters'])
        size = int(cfg_parser[section]['size'])
        stride = int(cfg_parser[section]['stride'])
        pad = int(cfg_parser[section]['pad'])
        activation = cfg_parser[section]['activation']
        batch_normalize = 'batch_normalize' in cfg_parser[section]

        padding = 'same' if pad == 1 and stride == 1 else 'valid'

        # Setting weights.
        # Darknet serializes convolutional weights as:
        # [bias/beta, [gamma, mean, variance], conv_weights]
        prev_layer_shape = K.int_shape(prev_layer)

        weights_shape = (size, size, prev_layer_shape[-1], filters)
        darknet_w_shape = (filters, weights_shape[2], size, size)
        weights_size = np.product(weights_shape)

        print('conv2d', 'bn' if batch_normalize else '  ', activation, weights_shape)

        conv_bias = np.ndarray(shape=(filters, ), dtype='float32',
                               buffer=weights_file.read(filters * 4))
        count += filters

        if batch_normalize:
            bn_weights = np.ndarray(shape=(3, filters), dtype='float32',
                                    buffer=weights_file.read(filters * 12))
            count += 3 * filters

            bn_weight_list = [
                bn_weights[0],  # scale gamma
                conv_bias,      # shift beta
                bn_weights[1],  # running mean
                bn_weights[2],  # running var
            ]

        conv_weights = np.ndarray(shape=darknet_w_shape,
                                  dtype='float32',
                                  buffer=weights_file.read(weights_size * 4))
        count += weights_size

        # DarkNet conv_weights are serialized Caffe-style:
        # (out_dim, in_dim, height, width)
        # We would like to set these to Tensorflow order:
        # (height, width, in_dim, out_dim)
        conv_weights = np.transpose(conv_weights, [2, 3, 1, 0])
        conv_weights = [conv_weights] if batch_normalize else [
            conv_weights, conv_bias
        ]

        # Create Conv2D layer
        if stride>1:
            # Darknet uses left and top padding instead of 'same' mode
            prev_layer = ZeroPadding2D(((1,0),(1,0)))(prev_layer)
        conv_layer = (Conv2D(
            filters, (size, size),
            strides=(stride, stride),
            kernel_regularizer=l2(weight_decay),
            use_bias=not batch_normalize,
            weights=conv_weights,
            activation=act_fn,
            padding=padding))(prev_layer)

        if batch_normalize:
            conv_layer = (BatchNormalization(
                weights=bn_weight_list))(conv_layer)
        prev_layer = conv_layer

        if activation == 'linear':
            all_layers.append(prev_layer)
        elif activation == 'leaky':
            act_layer = LeakyReLU(alpha=0.1)(prev_layer)
            prev_layer = act_layer
            all_layers.append(act_layer)

    #===============
    elif section.startswith('route'):
        ids = [int(i) for i in cfg_parser[section]['layers'].split(',')]
        layers = [all_layers[i] for i in ids]
        if len(layers) > 1:
            print('Concatenating route layers:', layers)
            concatenate_layer = Concatenate()(layers)
            all_layers.append(concatenate_layer)
            prev_layer = concatenate_layer
        else:
            skip_layer = layers[0]  # only one layer to route
            all_layers.append(skip_layer)
            prev_layer = skip_layer

    #===============
    elif section.startswith('maxpool'):
        size = int(cfg_parser[section]['size'])
        stride = int(cfg_parser[section]['stride'])
        all_layers.append(
            MaxPooling2D(
                pool_size=(size, size),
                strides=(stride, stride),
                padding='same')(prev_layer))
        prev_layer = all_layers[-1]

    #===============
    elif section.startswith('shortcut'):
        index = int(cfg_parser[section]['from'])
        activation = cfg_parser[section]['activation']
        assert activation == 'linear', 'Only linear activation supported.'
        all_layers.append(Add()([all_layers[index], prev_layer]))
        prev_layer = all_layers[-1]

    #===============
    elif section.startswith('upsample'):
        stride = int(cfg_parser[section]['stride'])
        assert stride == 2, 'Only stride=2 supported.'
        all_layers.append(UpSampling2D(stride)(prev_layer))
        prev_layer = all_layers[-1]

    #===============
    elif section.startswith('yolo'):
        out_index.append(len(all_layers)-1)
        all_layers.append(None)
        prev_layer = all_layers[-1]

    #===============
    elif section.startswith('net'):
        pass

    #===============
    else:
        raise ValueError('Unsupported section header type: {}'.format(section))

=== Parsing section : net_0 ===
=== Parsing section : convolutional_0 ===


NameError: name 'K' is not defined